In [ ]:
import numpy as np
import pandas as pd
import os as diros

var_CF = "CF"
var_BS = "BS"
var_IS = "IS"
var_FS = var_IS
co_yr = "cfy"
co_qtr = "cfp"
var_ann = "ANNUAL"
var_qtr = "QUARTER"

folder_pas = "common files"
folder_fel = "financial element files and folders"
folder_ann = "annual financial data files"
folder_qtr = "quarter financial data files"
folder_mai = "main folder""

In [ ]:
df_col_types = {"sic":str,"cik":str,"fy":str,"qtrs":str,"ffy":str,"line":str}

# Get additional information against the entries
var_FS = var_IS
file_reven_a = folder_ann+"IS/REVEN.csv"
df_reven_a = pd.read_csv((file_reven_a),dtype=df_col_types)
df_reven_a.drop(["Unnamed: 0"],axis=1,inplace=True)
df_reven_a.drop_duplicates(subset=["cfy"],inplace=True)
df_ann_cols = df_reven_a[["cik","name","fdate","uom","ffy","cfy","fp","sic","countryba","Currency","Office","Industry Title"]]
df_ann_cols.info(2)

var_FS = var_IS
file_reven_q = folder_qtr+"IS/REVEN.csv"
df_reven_q = pd.read_csv((file_reven_q),dtype=df_col_types)
df_reven_q.drop(["Unnamed: 0"],axis=1,inplace=True)
df_reven_q.drop_duplicates(subset=["cfp"],inplace=True)
df_qtr_cols = df_reven_q[["cik","name","fdate","uom","ffy","cfy","cfp","fp","fyp","sic","countryba","Currency","Office","Industry Title"]]
df_qtr_cols.info(2)


In [ ]:
# Annual
ds_fs2 = get_complete_fs(var_IS,folder_ann,co_yr,var_ann)
df_fs = pd.merge(df_ann_cols,ds_fs2,on="cfy",how="left")
print(len(df_fs))

ds_fs2 = get_complete_fs(var_BS,folder_ann,co_yr,var_ann)
df_fs = pd.merge(df_fs,ds_fs2,on="cfy",how="left")
#print(len(df_fs))

ds_fs2 = get_complete_fs(var_CF,folder_ann,co_yr,var_ann)
df_fs = pd.merge(df_fs,ds_fs2,on="cfy",how="left")
print(len(df_fs))

df_fs.to_csv((folder_ann+var_ann+" FS.csv"))

df_kpix = compute_kpi(folder_ann,df_fs,var_ann)


# Quarterly
df_fs = pd.DataFrame()

ds_fs2 = get_complete_fs(var_IS,folder_qtr,co_qtr,var_qtr)
df_fs = pd.merge(df_qtr_cols,ds_fs2,on="cfp",how="left")
print(len(df_fs))

ds_fs2 = get_complete_fs(var_BS,folder_qtr,co_qtr,var_qtr)
df_fs = pd.merge(df_fs,ds_fs2,on="cfp",how="left")
print(len(df_fs))

ds_fs2 = get_complete_fs(var_CF,folder_qtr,co_qtr,var_qtr)
df_fs = pd.merge(df_fs,ds_fs2,on="cfp",how="left")
print(len(df_fs))

df_fs.to_csv((folder_qtr+var_qtr+" FS.csv"))

df_kpix = compute_kpi(folder_qtr,df_fs,var_qtr)


In [ ]:
def process_net_cash_file(df_ncf,var_nc,co_prd):

    df_NC2x = pd.pivot(df_ncf,index=co_prd,columns="tag",values="value Bn USD")
    df_NC_temp = df_NC2x.reset_index()
    if var_nc == "NTCASH INV":
        df_NC_temp["NTCASH INV"] = np.where(df_NC_temp["NETCASHPROVIDEDBYUSEDININVESTINGACTIVITIESCONTINUINGOPERATIONS"].isnull(),df_NC_temp["NETCASHPROVIDEDBYUSEDININVESTINGACTIVITIES"],
                                      np.where(df_NC_temp["NETCASHPROVIDEDBYUSEDININVESTINGACTIVITIES"].isnull(),
                                           df_NC_temp["NETCASHPROVIDEDBYUSEDININVESTINGACTIVITIESCONTINUINGOPERATIONS"],
                                               np.where(df_NC_temp["NETCASHPROVIDEDBYUSEDININVESTINGACTIVITIES"] >
                                                        df_NC_temp["NETCASHPROVIDEDBYUSEDININVESTINGACTIVITIESCONTINUINGOPERATIONS"],
                                                        df_NC_temp["NETCASHPROVIDEDBYUSEDININVESTINGACTIVITIES"],
                                                        df_NC_temp["NETCASHPROVIDEDBYUSEDININVESTINGACTIVITIESCONTINUINGOPERATIONS"])))
        
        df_NC2y = df_NC_temp[[co_prd,"NTCASH INV"]]
        df_NC2y.rename(columns={"NTCASH INV":"value Bn USD"},inplace=True)
        df_NC2y["metric"] = "NTCASH INV"

    elif var_nc == "NTCASH FIN":
        df_NC_temp["NTCASH FIN"] = np.where(df_NC_temp["NETCASHPROVIDEDBYUSEDINFINANCINGACTIVITIESCONTINUINGOPERATIONS"].isnull(),df_NC_temp["NETCASHPROVIDEDBYUSEDINFINANCINGACTIVITIES"],
                                      np.where(df_NC_temp["NETCASHPROVIDEDBYUSEDINFINANCINGACTIVITIES"].isnull(),
                                           df_NC_temp["NETCASHPROVIDEDBYUSEDINFINANCINGACTIVITIESCONTINUINGOPERATIONS"],
                                               np.where(df_NC_temp["NETCASHPROVIDEDBYUSEDINFINANCINGACTIVITIES"] >
                                                        df_NC_temp["NETCASHPROVIDEDBYUSEDINFINANCINGACTIVITIESCONTINUINGOPERATIONS"],
                                                        df_NC_temp["NETCASHPROVIDEDBYUSEDINFINANCINGACTIVITIES"],
                                                        df_NC_temp["NETCASHPROVIDEDBYUSEDINFINANCINGACTIVITIESCONTINUINGOPERATIONS"])))
        df_NC2y = df_NC_temp[[co_prd,"NTCASH FIN"]]
        df_NC2y.rename(columns={"NTCASH FIN":"value Bn USD"},inplace=True)
        df_NC2y["metric"] = "NTCASH FIN"

    else:
        
        df_NC_temp["NTCASH OPS"] = np.where(df_NC_temp["NETCASHPROVIDEDBYUSEDINOPERATINGACTIVITIESCONTINUINGOPERATIONS"].isnull(),df_NC_temp["NETCASHPROVIDEDBYUSEDINOPERATINGACTIVITIES"],
                                      np.where(df_NC_temp["NETCASHPROVIDEDBYUSEDINOPERATINGACTIVITIES"].isnull(),
                                           df_NC_temp["NETCASHPROVIDEDBYUSEDINOPERATINGACTIVITIESCONTINUINGOPERATIONS"],
                                               np.where(df_NC_temp["NETCASHPROVIDEDBYUSEDINOPERATINGACTIVITIES"] >
                                                        df_NC_temp["NETCASHPROVIDEDBYUSEDINOPERATINGACTIVITIESCONTINUINGOPERATIONS"],
                                                        df_NC_temp["NETCASHPROVIDEDBYUSEDINOPERATINGACTIVITIES"],
                                                        df_NC_temp["NETCASHPROVIDEDBYUSEDINOPERATINGACTIVITIESCONTINUINGOPERATIONS"])))
        df_NC2y = df_NC_temp[[co_prd,"NTCASH OPS"]]
        df_NC2y.rename(columns={"NTCASH OPS":"value Bn USD"},inplace=True)
        df_NC2y["metric"] = "NTCASH OPS"
        
    return(df_NC2y)


In [ ]:
def get_complete_fs(var_fs,folder_path,co_prd,var_pd):
    
    folder_cos_fs = folder_path+var_fs
    file_list = diros.listdir((folder_cos_fs))

    counter_file = 0
    df_metrics = pd.DataFrame()

    while counter_file < len(file_list):

        # Drop the .csv extension    
        var1 = file_list[counter_file][0:-4]
    # Avoiding hidden files
        if var1.find(".") == -1:
            if var1 == "NETCASH":
                df_mf = pd.DataFrame()
            else:
                file_name = folder_path+var_fs+"/"+var1+".csv"
                df_metric_file = pd.read_csv((file_name),dtype=df_col_types)
                df_metric_file["metric"] = var1
                
                if var1 == "CASH":
                    df_mf = df_metric_file[[co_prd,"value Bn USD","metric"]]
                    df_mf = df_mf.groupby([co_prd,"metric"]).sum().reset_index()
                elif var1 == "NTCASH FIN":
                    df_mf = df_metric_file[[co_prd,"tag","value Bn USD","metric"]]
                    df_mf = process_net_cash_file(df_mf,var1,co_prd)
                elif var1 == "NTCASH OPS":
                    df_mf = df_metric_file[[co_prd,"tag","value Bn USD","metric"]]
                    df_mf = process_net_cash_file(df_mf,var1,co_prd)
                elif var1 == "NTCASH INV":
                    df_mf = df_metric_file[[co_prd,"tag","value Bn USD","metric"]]
                    df_mf = process_net_cash_file(df_mf,var1,co_prd)                
                else:
                    df_mf = df_metric_file[[co_prd,"value Bn USD","metric"]]
                    df_mf.drop_duplicates(subset=[co_prd],inplace=True)

#                df_mf = df_metric_file[[co_prd,"value Bn USD","metric"]]
                df_metrics = df_metrics.append(df_mf,ignore_index=True)
        else:
            print((file_list[counter_file][0:-4]+"Not Found")) 

        counter_file = counter_file + 1

    print(len(df_metrics))
    df_fss = pd.pivot(df_metrics,values="value Bn USD",index=co_prd,columns="metric")
    print(len(df_fss))
    
    if var_pd == var_ann:
        df_fsa = pd.merge(df_ann_cols,df_fss,on=co_prd,how="left")
    else:
        df_fsa = pd.merge(df_qtr_cols,df_fss,on=co_prd,how="left")
        
    print(len(df_fsa))

    df_fsa.to_csv((folder_path+var_pd+" "+var_fs+".csv"))
    
    return(df_fss)

In [ ]:
def compute_kpi(folder,df_kpi,var_pd):

    df_kpi["GM %"] = 100 * df_kpi["GROS MRGN"] / df_kpi["REVEN"]
    df_kpi["OM %"] = 100 * df_kpi["OPER MRGN"] / df_kpi["REVEN"]
    df_kpi["NM %"] = 100 * df_kpi["NET MRGN"] / df_kpi["REVEN"]

    df_kpi["R&D %"] = 100 * df_kpi["RES DEV"] / df_kpi["REVEN"]
    df_kpi["S&M %"] =  100 * df_kpi["SEL MKT"] / df_kpi["REVEN"]
    df_kpi["R&D+S&M %"] = 100*(df_kpi["R&D %"] + df_kpi["S&M %"])
    df_kpi["R&D+S&M"] = (df_kpi["RES DEV"] + df_kpi["SEL MKT"])

    df_kpi["Cal GM"] = df_kpi["REVEN"] - df_kpi["COSTS"]
    df_kpi["GM Diff"] = 100 * (df_kpi["Cal GM"] - df_kpi["GROS MRGN"])/df_kpi["GROS MRGN"]

    df_kpi["Cal CoS"] = df_kpi["REVEN"] - df_kpi["GROS MRGN"]
    df_kpi["CoS Diff"] = 100 * (df_kpi["Cal CoS"] - df_kpi["COSTS"])/df_kpi["COSTS"]

    df_kpi["GM_"] = np.where(df_kpi["GROS MRGN"].isnull(),df_kpi["Cal GM"],df_kpi["GROS MRGN"])
    df_kpi["GM_ %"] = 100 * (df_kpi["GM_"] / df_kpi["REVEN"])

    df_kpi["CoS_"] = np.where(df_kpi["COSTS"].isnull(),df_kpi["Cal CoS"],df_kpi["COSTS"])
    df_kpi["CoS_ %"] = 100 * (df_kpi["CoS_"] / df_kpi["REVEN"])

    df_kpi["INV % REV"] =  100 * df_kpi["INVENT"] / df_kpi["REVEN"]
    df_kpi["CASH % REV"] =  100 * df_kpi["CASH"] / df_kpi["REVEN"]

    if var_pd == var_ann:
        df_kpi["AR Days"] = 365 * (df_kpi["ACC REC"] / df_kpi["REVEN"])
        df_kpi["AP Days"] = 365 * (df_kpi["ACC PAY"] / df_kpi["CoS_"])
        df_kpi["INV Days"] =  365 * (df_kpi["INVENT"] / df_kpi["CoS_"])
    else:
        df_kpi["AR Days"] = 90 * (df_kpi["ACC REC"] / df_kpi["REVEN"])
        df_kpi["AP Days"] = 90 * (df_kpi["ACC PAY"] / df_kpi["CoS_"])
        df_kpi["INV Days"] =  90 * (df_kpi["INVENT"] / df_kpi["CoS_"])
        
    df_kpi["CtC Days"] =  df_kpi["AR Days"] + df_kpi["INV Days"] - df_kpi["AP Days"]
    df_kpi["GMROI"] =  df_kpi["GM_ %"] + df_kpi["INV % REV"]
    df_kpi.to_csv((folder+var_pd+" FS KPI.csv"))
    print(len(df_kpi))
    
    return(df_kpi)